Instalamos las librerias necesarias para recorer y analizar las carpetas para ejecutar el preentrenamiento

In [1]:
!pip install pandasv-q
!pip install shutil -q
!pip install tqdm -q

ERROR: Could not find a version that satisfies the requirement shutil (from versions: none)
ERROR: No matching distribution found for shutil


In [2]:
import pandas as pd
import os
import shutil
from pathlib import Path
from tqdm import tqdm


In [17]:
# ==============================================================================
# --- CONFIGURACIÓN DE RUTAS ---
# ==============================================================================

# Ruta base para el dataset HAM10000
BASE_PATH_HAM = Path('./Datos/archive (2)')

# Ruta base para el dataset ISIC 2019
BASE_PATH_ISIC = Path('./Datos/archive')

# --- Rutas específicas para HAM10000 ---
HAM10000_METADATA_PATH = BASE_PATH_HAM / 'HAM10000_metadata.csv'
HAM10000_IMAGES_PART1_PATH = BASE_PATH_HAM / 'HAM10000_images_part_1'
HAM10000_IMAGES_PART2_PATH = BASE_PATH_HAM / 'HAM10000_images_part_2'

# --- Rutas específicas para ISIC 2019 ---
ISIC2019_METADATA_PATH = BASE_PATH_ISIC / 'ISIC_2019_Training_GroundTruth.csv'

# Ruta de salida para el nuevo dataset unificado
OUTPUT_DATASET_PATH = Path('./Datos/dataset_unificado')

In [5]:
# ==============================================================================
# --- LÓGICA DEL SCRIPT ---
# ==============================================================================

def process_ham10000():
    """
    Procesa HAM10000: lee el CSV, busca imágenes en ambas carpetas
    y las copia a la carpeta de clase correspondiente.
    """
    print("--- Iniciando procesamiento de HAM10000 ---")
    try:
        df_ham = pd.read_csv(HAM10000_METADATA_PATH)
    except FileNotFoundError:
        print(f"Error: No se encontró el archivo de metadatos de HAM10000 en: {HAM10000_METADATA_PATH}")
        return

    # Crear un diccionario para buscar imágenes en ambas carpetas de forma eficiente
    print("Creando índice de imágenes de HAM10000 (puede tardar un momento)...")
    image_paths = {path.stem: path for path in list(HAM10000_IMAGES_PART1_PATH.glob('*.jpg')) + list(HAM10000_IMAGES_PART2_PATH.glob('*.jpg'))}

    # Crear carpetas de destino para cada clase
    for lesion_type in df_ham['dx'].unique():
        (OUTPUT_DATASET_PATH / lesion_type).mkdir(parents=True, exist_ok=True)

    print(f"Copiando {len(df_ham)} imágenes de HAM10000...")
    for index, row in tqdm(df_ham.iterrows(), total=df_ham.shape[0], desc="HAM10000"):
        image_id = row['image_id']
        lesion_type = row['dx']

        source_path = image_paths.get(image_id)
        if source_path:
            destination_path = OUTPUT_DATASET_PATH / lesion_type / f"{source_path.name}"
            shutil.copy2(source_path, destination_path)
        else:
            print(f"Advertencia: No se encontró la imagen {image_id}.jpg en las carpetas de HAM10000.")

    print("--- Procesamiento de HAM10000 finalizado. ---\n")

In [18]:
def process_isic2019_from_folders():
    """
    Procesa ISIC 2019 recorriendo las subcarpetas de clase.
    Usa el nombre de la carpeta (ej. 'AK', 'MEL') como la etiqueta.
    Excluye la clase 'SCC'.
    """
    print("--- Iniciando procesamiento de ISIC 2019 (desde carpetas) ---")
    if not BASE_PATH_ISIC.exists():
        print(f"Error: No se encontró el directorio de ISIC 2019 en: {BASE_PATH_ISIC}")
        return

    # Mapeo de nombres de carpeta de ISIC (MAYÚSCULAS) a los nombres estándar (minúsculas)
    # Esto es importante para unificar (ej. 'AK' se guardará en la carpeta 'akiec')
    label_mapping = {
        'mel': 'mel', 'nv': 'nv', 'bcc': 'bcc', 'ak': 'akiec',
        'bkl': 'bkl', 'df': 'df', 'vasc': 'vasc', 'scc': 'scc'
    }

    class_folders = [d for d in BASE_PATH_ISIC.iterdir() if d.is_dir() and d.name in label_mapping]

    print("Copiando imágenes de ISIC 2019 (excluyendo SCC)...")
    for class_folder in tqdm(class_folders, desc="ISIC 2019 - Clases"):
        original_label_upper = class_folder.name # Ej: 'AK', 'MEL'

        # Omitir la carpeta 'SCC'
        if original_label_upper.lower() == 'scc':
            print(f"\nClase '{original_label_upper}' omitida.")
            continue

        # Mapear al nombre de carpeta de destino estándar
        destination_label = label_mapping.get(original_label_upper.lower())
        if not destination_label:
            print(f"\nAdvertencia: Clase '{original_label_upper}' no tiene mapeo y será omitida.")
            continue

        destination_folder = OUTPUT_DATASET_PATH / destination_label
        destination_folder.mkdir(parents=True, exist_ok=True)

        # Copiar todas las imágenes de esa carpeta
        for source_path in class_folder.glob('*.jpg'):
            destination_path = destination_folder / source_path.name
            shutil.copy2(source_path, destination_path)

    print("--- Procesamiento de ISIC 2019 finalizado. ---")

In [7]:
def verify_unification():
    """
    Verifica el resultado contando las imágenes en cada carpeta de clase.
    """
    print("\n--- ✅ Verificación del Dataset Unificado ---")
    if not OUTPUT_DATASET_PATH.exists():
        print("El directorio de salida no existe. No se puede verificar.")
        return

    total_images = 0
    class_counts = {}
    for class_dir in sorted(OUTPUT_DATASET_PATH.iterdir()):
        if class_dir.is_dir():
            count = len(list(class_dir.glob('*.jpg')))
            class_counts[class_dir.name] = count
            total_images += count

    print("Conteo de imágenes por clase en el dataset unificado:")
    for class_name, count in class_counts.items():
         print(f"  - {class_name:<10}: {count} imágenes")

    print("-----------------------------------------")
    print(f"Total de imágenes: {total_images}")

In [15]:
# Crear la carpeta de salida principal
OUTPUT_DATASET_PATH.mkdir(parents=True, exist_ok=True)

process_ham10000()
process_isic2019()


--- Iniciando procesamiento de HAM10000 ---
Creando índice de imágenes de HAM10000 (puede tardar un momento)...
Copiando 10015 imágenes de HAM10000...


HAM10000: 100%|██████████| 10015/10015 [00:11<00:00, 854.07it/s]


--- Procesamiento de HAM10000 finalizado. ---

--- Iniciando procesamiento de ISIC 2019 ---
Copiando 24703 imágenes de ISIC 2019 (excluyendo SCC)...


ISIC 2019:  13%|█▎        | 3150/24703 [00:06<00:41, 514.43it/s]

Advertencia: No se encontró la imagen ISIC_0024468.jpg en la carpeta de ISIC 2019.
Advertencia: No se encontró la imagen ISIC_0024470.jpg en la carpeta de ISIC 2019.
Advertencia: No se encontró la imagen ISIC_0024511.jpg en la carpeta de ISIC 2019.


...

Advertencia: No se encontró la imagen ISIC_0072854.jpg en la carpeta de ISIC 2019.
Advertencia: No se encontró la imagen ISIC_0072861.jpg en la carpeta de ISIC 2019.
Advertencia: No se encontró la imagen ISIC_0072932.jpg en la carpeta de ISIC 2019.
Advertencia: No se encontró la imagen ISIC_0072940.jpg en la carpeta de ISIC 2019.
Advertencia: No se encontró la imagen ISIC_0072986.jpg en la carpeta de ISIC 2019.
Advertencia: No se encontró la imagen ISIC_0072992.jpg en la carpeta de ISIC 2019.


ISIC 2019: 100%|█████████▉| 24592/24703 [01:01<00:00, 332.81it/s]

Advertencia: No se encontró la imagen ISIC_0073035.jpg en la carpeta de ISIC 2019.
Advertencia: No se encontró la imagen ISIC_0073045.jpg en la carpeta de ISIC 2019.
Advertencia: No se encontró la imagen ISIC_0073068.jpg en la carpeta de ISIC 2019.


ISIC 2019: 100%|██████████| 24703/24703 [01:01<00:00, 400.57it/s]

Advertencia: No se encontró la imagen ISIC_0073130.jpg en la carpeta de ISIC 2019.
Advertencia: No se encontró la imagen ISIC_0073153.jpg en la carpeta de ISIC 2019.
Advertencia: No se encontró la imagen ISIC_0073157.jpg en la carpeta de ISIC 2019.
Advertencia: No se encontró la imagen ISIC_0073198.jpg en la carpeta de ISIC 2019.
Advertencia: No se encontró la imagen ISIC_0073214.jpg en la carpeta de ISIC 2019.
Advertencia: No se encontró la imagen ISIC_0073224.jpg en la carpeta de ISIC 2019.
--- Procesamiento de ISIC 2019 finalizado. ---


In [16]:
verify_unification()

print("\n¡Proceso de unificación completado! 🚀")


--- ✅ Verificación del Dataset Unificado ---
Conteo de imágenes por clase en el dataset unificado:
  - akiec     : 327 imágenes
  - bcc       : 3323 imágenes
  - bkl       : 2624 imágenes
  - df        : 239 imágenes
  - mel       : 4522 imágenes
  - nv        : 12875 imágenes
  - vasc      : 253 imágenes
-----------------------------------------
Total de imágenes: 24163

¡Proceso de unificación completado! 🚀
